# Methode Exacte : Branch & Bound

Nous proposons un algorithme exacte de resolution du probleme de flow shop de permutation. selon les criteres suivants

## Strategie de Recherche

L'algorithme est parametrable par la strategie de recherche . On propose 2 strategie:

    - Recherche en profondeur : Depth First Search
    - Recherche Meilleur D'abord : Best First Search
selon l'instance les 2 algoritmes peuvent donner des resultats different parfois meme tres performants. 

## Espace de recherche

![branch and bound tree](./images/bnb.png)

# Evaluation des noeuds 

** Evaluation noeud Interne ** : calculer le Makespan $C^*_{max}$ de la sous sequence scheduled de ce noeud

** Evaluation feuille ** : calculer le Makespan total $C_{max}$

# Elagage des noeud interne :

Un noeud interne est élagué et donc non exploré si est seuelement si : 
$$
C^*_{max} > UpperBound
$$

In [1]:
import sys
sys.path.append("../") # in order to import fsp
from fsp import branch_and_bound
import matplotlib.pyplot as plt
from utils import Instance, Benchmark
import numpy as np

In [2]:
import plotly.figure_factory as ff
from datetime import datetime
import numpy as np
import plotly.express as px
instance4 = Instance(
    np.array([
        [1,2,3,2],
        [1,4,2,10],
        [3,2,1,5],
        [4,10,3,1],
        [1,5,4,4],
        [2,3,2,6],
        [5,2,1,1],
        [2,3,2,6],
        [5,2,1,1],
    ], dtype=np.int64)
)
results = branch_and_bound.get_results(instance4,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=True,log=False)
print(results)
res = instance4.get_chart_data(results)

fig = px.timeline(res['df'], x_start="Start", x_end="Finish", y="Task", color="Resource")
fig.layout.xaxis.update({
        'tickvals' : res['date_ticks'],
        'ticktext' : res['num_tick_labels']
        })
fig.show()

start seq(0, 6, 8, 2, 5, 7, 4, 1, 3)
upper bound 49.0
{'C_max': 42.0, 'order': [0, 2, 1, 4, 5, 8, 3, 7, 6], 'details': {'explored': 229926, 'pruned': 180532, 'leafs': 4, 'time': 7.348183800000015}}


In [3]:
# without heuristique init (uses starting sequence)
results2 = branch_and_bound.get_results(instance4,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=False,log=False)
print(results2)
res2 = instance4.get_chart_data(results2)

fig2 = px.timeline(res2['df'], x_start="Start", x_end="Finish", y="Task", color="Resource")
fig2.layout.xaxis.update({
        'tickvals' : res['date_ticks'],
        'ticktext' : res['num_tick_labels']
        })
fig2.show()

start seq(0, 1, 2, 3, 4, 5, 6, 7, 8)
upper bound 46.0
{'C_max': 42.0, 'order': [0, 2, 1, 4, 5, 8, 3, 7, 6], 'details': {'explored': 229925, 'pruned': 180532, 'leafs': 3, 'time': 7.086145500000043}}


# Depth-First-Search vs Best-First-Search

In [7]:
jobs = 9
machines = 4
random_mat = np.random.random((jobs,machines)) * 100
randomInstance = Instance(random_mat)
random_mat

array([[59.32178504, 49.89094722, 12.66912934,  2.87980327],
       [50.4168833 , 69.69190645, 62.27260146, 15.65760823],
       [42.21630933, 61.10997132,  3.44555249, 98.77285774],
       [82.92433093, 89.54121788, 80.76033336, 89.67184254],
       [ 1.9164287 , 35.72980973,  9.86583552, 89.78697495],
       [86.68715181, 31.06685406, 89.34428901, 42.20724588],
       [ 8.74755206, 55.57178241, 74.80407291, 14.78184065],
       [88.10323181, 45.49933266, 71.81924293, 22.81962431],
       [39.91595238, 86.11028384, 90.26361293, 38.03924651]])

In [8]:
depth_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=False,log=False)
depth_res

start seq(0, 1, 2, 3, 4, 5, 6, 7, 8)
upper bound 784.9783971505342


{'C_max': 595.7713483513661,
 'order': [4, 6, 8, 2, 5, 3, 7, 1, 0],
 'details': {'explored': 177310,
  'pruned': 194138,
  'leafs': 37,
  'time': 5.661664400000063}}

In [9]:
best_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.BEST_FIRST_SEARCH,use_heuristique_init=False,log=False)
best_res

start seq(0, 1, 2, 3, 4, 5, 6, 7, 8)
upper bound 784.9783971505342


{'C_max': 595.7713483513661,
 'order': [4, 6, 8, 2, 5, 3, 7, 1, 0],
 'details': {'explored': 112432,
  'pruned': 165954,
  'leafs': 25,
  'time': 5.364031299999965}}

# Impact de la distribution initiale des couts sur les machines

## Instance a couts suivant une loi normale 

In [10]:
jobs = 9
machines = 3
mean_time = 10
std_time = 20
random_mat = np.abs(np.random.normal(loc=mean_time,scale=std_time,size=(jobs,machines)))
randomInstance = Instance(random_mat)
random_mat

array([[3.17895194e+01, 1.41193214e-02, 2.83976599e+00],
       [7.23513193e+00, 2.10987188e+01, 1.67693662e+01],
       [1.18754110e+01, 1.34906319e+00, 3.00011517e+00],
       [5.56875803e+00, 9.17744929e+00, 1.97832377e+01],
       [2.44439047e+00, 6.47108760e+01, 3.45870075e+00],
       [2.58913784e+00, 1.74195210e+01, 2.48509364e+01],
       [8.10115244e+00, 1.43672789e+01, 1.60574067e+01],
       [2.93281635e+01, 3.15073228e+01, 4.68810063e-01],
       [4.88943353e+01, 2.67043206e+01, 2.30721040e+00]])

In [11]:
heur_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=True,log=False)
heur_res

start seq(2, 3, 0, 6, 5, 1, 7, 4, 8)
upper bound 235.45008932471066


{'C_max': 189.26187041168768,
 'order': [4, 0, 1, 2, 3, 5, 8, 6, 7],
 'details': {'explored': 239989,
  'pruned': 156899,
  'leafs': 23,
  'time': 5.340898500000094}}

In [12]:
noheur_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=False,log=False)
noheur_res

start seq(0, 1, 2, 3, 4, 5, 6, 7, 8)
upper bound 227.66641225715108


{'C_max': 189.26187041168768,
 'order': [4, 0, 1, 2, 3, 5, 8, 6, 7],
 'details': {'explored': 239989,
  'pruned': 156899,
  'leafs': 23,
  'time': 6.010924100000011}}

## Instance a couts suivant une loi uniforme

In [29]:
jobs = 9
machines = 4
low = 10
high = 100
random_mat = np.abs(np.random.uniform(low=low,high=high,size=(jobs,machines)))
randomInstance = Instance(random_mat)
random_mat

array([[28.75948187, 30.08885274, 88.61330802, 12.54000721],
       [65.53163835, 26.94485821, 17.46386911, 42.17436021],
       [55.3295282 , 47.18536725, 83.15180349, 63.55754009],
       [60.88998033, 56.54074946, 51.14439568, 86.64426659],
       [70.42342534, 42.57919888, 26.74380321, 88.04871627],
       [58.73137839, 61.47333225, 65.98010658, 34.38243297],
       [94.09172985, 36.26401766, 57.74166741, 30.95906234],
       [78.51757727, 42.06067473, 98.02980989, 73.30842163],
       [41.05458764, 75.66953873, 50.29092212, 88.16854389]])

In [30]:
heur_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=True,log=False)
heur_res

start seq(1, 0, 6, 5, 4, 2, 8, 3, 7)
upper bound 814.8831249390905


{'C_max': 686.7983997001883,
 'order': [8, 2, 3, 7, 5, 4, 6, 1, 0],
 'details': {'explored': 260601,
  'pruned': 255090,
  'leafs': 23,
  'time': 13.835180400000354}}

In [31]:
noheur_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=False,log=False)
noheur_res

start seq(0, 1, 2, 3, 4, 5, 6, 7, 8)
upper bound 813.842189746039


{'C_max': 686.7983997001883,
 'order': [8, 2, 3, 7, 5, 4, 6, 1, 0],
 'details': {'explored': 260601,
  'pruned': 255090,
  'leafs': 23,
  'time': 37.96600939999962}}

## Instance a cout de loi exponentielle

In [44]:
jobs = 9
machines = 4
scale = 10
random_mat = np.random.exponential(scale=scale,size=(jobs,machines))
randomInstance = Instance(random_mat)
random_mat

array([[ 8.67850222,  4.3264421 , 20.29488751, 40.21384655],
       [30.59318083, 12.94590949, 19.17144682,  1.39382758],
       [ 6.69261742,  0.91875524,  0.733197  , 11.09468333],
       [ 7.76209271,  0.91041036,  0.64665727,  7.48669551],
       [ 4.84153552,  4.64264686,  4.49365645, 11.58530051],
       [12.63722871,  2.63389906, 17.63252946, 24.74045613],
       [ 9.07937578,  7.84126981, 14.01722019,  0.2725492 ],
       [17.74234497,  1.74945508, 11.02846333,  0.09056551],
       [16.63812413, 11.74067692,  3.12347859,  8.24455906]])

In [45]:
heur_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=True,log=False)
heur_res

start seq(3, 2, 4, 7, 6, 8, 5, 1, 0)
upper bound 198.61259043868293


{'C_max': 127.53348621223064,
 'order': [2, 3, 4, 5, 0, 6, 1, 8, 7],
 'details': {'explored': 124236,
  'pruned': 138278,
  'leafs': 6,
  'time': 7.162785700000313}}

In [47]:
noheur_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=False,log=False)
noheur_res

start seq(0, 1, 2, 3, 4, 5, 6, 7, 8)
upper bound 138.42231522023357


{'C_max': 127.53348621223064,
 'order': [2, 3, 4, 5, 0, 6, 1, 8, 7],
 'details': {'explored': 124235,
  'pruned': 138278,
  'leafs': 5,
  'time': 7.083648699999685}}